In [1]:
import aoc
import numpy as np
from scipy.ndimage import convolve
import math
import re
import itertools
from functools import reduce
import networkx as nx
import matplotlib.pyplot as plt

# Day 3

In [ ]:
raw_data = aoc.get_data(2020, 3)

In [ ]:
data = raw_data.replace('.', '0').replace('#', '1').splitlines()

In [ ]:
data

In [ ]:
patch = np.array([[int(y) for y in x] for x in data])

In [ ]:
slope = (3, 1)

In [ ]:
def make_forest(patch, slope):
    
    shape = patch.shape
    
    max_down = shape[0] * slope[1]
    
    max_right = max_down * slope[0]
    
    return np.concatenate([patch] * math.ceil(max_right / shape[1]), axis=1)

In [ ]:
forest = make_forest(patch, (1, 3))

In [ ]:
forest.shape

In [ ]:
def count_trees(forest, slope):
    
    x = 0
    y = 0
    
    trees = []
    
    while True:
        try: 
            x += slope[0]
            y += slope[1]
        
            trees.append(forest[y, x])
        except IndexError:
            break
    
    return sum(trees)

In [ ]:
count_trees(forest, slope)

# Day 4

In [ ]:
raw_data = aoc.get_data(2020, 4)

In [ ]:
passports = raw_data.split('\n\n')

In [ ]:
test = """eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007""".split('\n\n')

In [ ]:
test

In [ ]:
required_keys = {'byr', 'iyr', 
'eyr' ,
'hgt' ,
'hcl' ,'ecl', 
'pid' ,
}

In [ ]:
required_keys

In [ ]:
def validate_all_passport_fields(passport):
    
    items = passport.replace(' ', '\n').splitlines()
    
    document = dict()
    
    for item in items:
    
        key, value = item.split(':')
        
        document[key] = value
    
    if set(document.keys()).intersection(required_keys) == required_keys:
        
        return document
    
    
def valid_between(value, low, high):
    
    value = int(value)
    
    return low <= value <= high


def valid_height(value):

    if re.search('\d*in', value) is not None:
        
        return valid_between(value[:-2], 59, 76)
    
    elif re.search('\d*cm', value) is not None:
        
        return valid_between(value[:-2], 150, 193)
        
    else:
        return False

def valid_hair(value):

    return re.search('#[0-9a-f]{6}', value) is not None

def valid_eye(value):

    return value in {'amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth'}

def valid_passport_number(value):
    
    return re.search('^\d{9}$', value) is not None

    
def validate_passport(passport):
    
    document = validate_all_passport_fields(passport)
    
    if document:
        
        return all([
            valid_between(document['byr'], 1920, 2002),
            valid_between(document['iyr'], 2010, 2020),
            valid_between(document['eyr'], 2020, 2030),
            valid_height(document['hgt']),
            valid_hair(document['hcl']), 
            valid_eye(document['ecl']), 
            valid_passport_number(document['pid'])])
    else:
        return False
            
        

In [ ]:
sum(map(lambda x: x is not None , map(validate_all_passport_fields, passports)))

In [ ]:
sum(map(validate_passport, passports))

# Day 5

In [ ]:
raw_data = aoc.get_data(2020, 5)

In [ ]:
data = raw_data.splitlines()

In [ ]:
def search(string, array, low):
    
    array_length = len(array)
    
    if array_length == 1:
        return array[0]
    
    comparitor = [*string].pop(0)
    
    if comparitor == low:
        
        return search(string[1:], array[: array_length//2], low)
    else:
        
        return search(string[1:], array[array_length//2:], low)

In [ ]:
def get_seat(code):
    
    row_code = code[:-3]
    col_code = code[-3:]
    
    row = search(row_code, list(range(128)), 'F')
    col = search(col_code, list(range(8)), 'L')
    
    return row * 8 + col
    

In [ ]:
seats = set(map(get_seat, data))

In [ ]:
max(seats)

In [ ]:
all_seats = set(range(min(seats), max(seats)))

In [ ]:
all_seats.difference(seats)

# Day 6 

In [ ]:
raw_data = aoc.get_data(2020, 6)

In [ ]:
data = raw_data.split('\n\n')

In [ ]:
def count_unique_questions(group):
    
    return len(set(group.replace('\n', '')))

In [ ]:
sum(map(count_unique_questions, data))

In [ ]:
def count_common_questions(group):
    
    people = [set(x) for x in group.splitlines()]
    
    return len(reduce(lambda a,b: a.intersection(b) , people))

In [ ]:
sum(map(count_common_questions, data))

# Day 7

In [ ]:
raw_data = aoc.get_data(2020,7)

In [ ]:
data = raw_data.splitlines()

In [ ]:
def make_bag_dict(bag_rules):
    
    bags = dict()
    
    for rule in bag_rules:
        
        color, contents = rule.replace('.', '').split(' bags contain ')
    
        contents = contents.split(', ')
        
        parsed_contents = list()
        
        for content in contents:
        
            content = re.search('\d+\s(.*)\sbags?', content)
            
            if content is None:
                
                bags[color] = None
            
            else:
                parsed_contents.append(content.group(1))
        
        bags[color] = parsed_contents
        
    return bags


In [ ]:
test = """light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags."""

In [ ]:
rules = make_bag_dict(data)
len(rules.keys())

In [ ]:
target = 'shiny gold'

In [ ]:
rules

In [ ]:
def find_target_bag(bag, target):
    
    contents = rules[bag]
    
    if type(contents) == bool:
        return contents
    
    if target in contents:
        
        rules[bag] = True
        
        return True
    else:
        rules[bag] = any([find_target_bag(x, target) for x in contents])
    

In [ ]:
for bag in rules.keys():
    find_target_bag(bag, target)

In [ ]:
sum(rules.values())

In [ ]:
def construct_rule_network(rules):
    
    graph = nx.DiGraph()
    
    for rule in rules:
        
        color, contents = rule.replace('.', '').split(' bags contain ')
        
        color = re.findall('^(.*?)\sbags?', rule)
        
        contents = re.findall('\d+\s(.*?)\sbags?', rule)
    
        if contents:
            
            graph.add_edges_from(zip(color*len(contents), contents))
        
        
    return graph
    

In [ ]:
graph = construct_rule_network(data)

In [ ]:
pos = nx.nx_agraph.pygraphviz_layout(graph, prog='dot')

fig, ax = plt.subplots(figsize=(16,9))
nx.draw_networkx(graph, pos, ax=ax)

In [ ]:
sum(map(lambda x: nx.has_path(graph, x, target) if x != target else False, list(graph.nodes)))

In [ ]:
def construct_weighted_rule_network(rules):
    
    g = nx.DiGraph()
    
    for rule in rules:
        
        color, contents = rule.replace('.', '').split(' bags contain ')
        
        color = re.findall('^(.*?)\sbags?', rule)
        
        contents = re.findall('\d+\s(.*?)\sbags?', rule)
        
        number = map(int, re.findall('(\d+)\s.*?\sbags?', rule))
    
        if contents:
            
            g.add_weighted_edges_from(zip(color*len(contents), contents, number))
        
        
    return g

In [ ]:
graph = construct_weighted_rule_network(data)

In [ ]:
decendents = nx.descendants(graph, target)

decendents.add(target)

sub_graph = graph.subgraph(decendents)

In [ ]:
pos = nx.nx_agraph.pygraphviz_layout(sub_graph, prog='dot')

fig, ax = plt.subplots(figsize=(16,9))
nx.draw_networkx(sub_graph, pos, ax=ax)

In [ ]:
def count_bag_contents(start):
    
    
    decendants = nx.descendants_at_distance(sub_graph, start, 1)
    
    if not decendants:
        return 1
    
    return sum(count_bag_contents(decendant) * sub_graph[start][decendant]['weight'] for decendant in decendants) + 1
        
    
    
    

In [ ]:
count_bag_contents('shiny gold') - 1

# Day 8

In [ ]:
raw_data = aoc.get_data(2020,8)

In [ ]:
data = raw_data.splitlines()

In [ ]:
class Console:
    
    
    def __init__(self):
        
        self.accumulator = 0
        self.pointer = 0
        self.instructions = set()
        self.data = None
        
        
    def run(self, data):
    
        self.accumulator = 0
        self.pointer = 0
        self.instructions = set()
        self.data = data
        
        while self.pointer not in self.instructions:
            
            self.next_instruction()
            
        return self.accumulator
    
    
    
    def next_instruction(self):
        
        self.instructions.add(self.pointer)
        
        instruction, value = self.data[self.pointer].split(' ')
        
        match instruction:
            
            case 'nop':
                
                self.pointer += 1
                
            case 'acc':
                self.accumulator += int(value)
                self.pointer += 1
                
            case 'jmp':
                self.pointer += int(value)
        

In [ ]:
console = Console()

In [ ]:
console.run(data)

In [ ]:
def make_new_tape(tape):
    
    for i in range(len(tape)):
                       
        verb = tape[i]
        new_tape = tape.copy()
        
        
        if 'nop' in verb:
        
            new_tape[i] = verb.replace('nop', 'jmp')
            
            
            
        elif 'jmp' in verb:
            
            new_tape[i] = verb.replace('jmp', 'nop')
            
                      
        yield new_tape.copy()
    

In [ ]:
a = make_new_tape(data)

In [ ]:
for tape in make_new_tape(data):
    
    try:
        console.run(tape)
    except IndexError:
        print(console.accumulator)

# Day 9

In [ ]:
raw_data = aoc.get_data(2020,9)

In [ ]:
data = list(map(int, raw_data.splitlines()))

In [ ]:
list(map(sum, itertools.permutations(data[0:25],2)))

In [ ]:
def find_invalid_number(data):
    
    preamble = data[0:25]
    
    check_values = data[25:]
    
    
    for value in check_values:
        
        if value not in list(map(sum, itertools.permutations(preamble,2))):
        
            return value
        
        preamble.pop(0)
        preamble.append(value)
        

In [ ]:
find_invalid_number(data)

In [ ]:
def find_invalid_number_with_preamble(data):
    
    preamble = data[0:25]
    
    check_values = data[25:]
    
    
    for value in check_values:
        
        if value not in list(map(sum, itertools.permutations(preamble,2))):
        
            return value, preamble
        
        preamble.pop(0)
        preamble.append(value)

In [ ]:
value, preamble = find_invalid_number_with_preamble(data)

In [ ]:
def sliding_window(data, window_size):
    
    
    for i in range(len(data) - window_size + 1):
        
        yield data[i:i+window_size]

In [ ]:
list(sliding_window(preamble, 3))

In [ ]:
def find_values_in_preamble(value, preamble):
    
    for window_size in range(3, len(preamble)+1):
                        
        for window in sliding_window(preamble, window_size):
            
            print(sum(window))
            
            if sum(window) == value:
                
                return min(window) + max(window)
    

In [ ]:
find_values_in_preamble(value, preamble)

# Day 10

In [ ]:
raw_data = aoc.get_data(2020, 10)

In [ ]:
data = list(map(int, raw_data.splitlines()))

In [ ]:
max(data)

In [ ]:
adaptor_joltage = max(data) + 3

In [ ]:
joltages = [0] + sorted(data) + [adaptor_joltage]

In [ ]:
joltages

In [ ]:
def diff_1(value):
    
    return value[1] - value[0] == 1


def diff_3(value):
    
    return value[1] - value[0] == 3


In [ ]:
sum(map(diff_1, aoc.sliding_window(joltages, 2))) * sum(map(diff_3, aoc.sliding_window(joltages, 2)))

In [ ]:
sum(map(diff_1, aoc.sliding_window(joltages, 2)))

# Day 11

In [ ]:
raw_data = aoc.get_data(2020, 11)

In [ ]:
raw_data.splitlines()

In [ ]:
data = np.array([[y for y in x ] for x in raw_data.splitlines()])

In [ ]:
data

In [ ]:
def fill_seat_rule(seat):
    
    # seat will be filled if it is empty already and no neigbhors
    
    return '#' not in seat and seat[1,1] != '.'

In [ ]:
def vacate_seat_rule(seat):

    filled = seat[1,1] == '#'
    
    surrounding = (seat == "#").sum() > 4

    return filled and surrounding
    
    

In [ ]:
def get_seat(seats, row, col):
    
    return seats[row - 1: row+2, col-1:col+2]

In [ ]:
def next_turn(seats_grid):
    
    pad = np.pad(seats_grid, 1, constant_values='.')
    
    update = seats_grid.copy()
    
    for row in range(seats.shape[0]):
        
        for col in range(seats.shape[1]):
            
            seat = get_seat(pad, row+1, col+1)

            
            if fill_seat_rule(seat):
                
                update[row, col] = '#'
    
            elif vacate_seat_rule(seat):
                update[row, col] = 'L'
                
                
    return update

In [ ]:
first = data
second = next_turn(data)


while not np.array_equal(first, second):
    
    first, second = second, next_turn(second) 
    

In [ ]:
(first == '#').sum()

In [ ]:
(test == "#").sum() > 4

In [ ]:
test

In [ ]:
def get_raycast_seat(seats, row, col):
    
    
    

# Day 12

In [4]:
raw_data = aoc.get_data(2020, 12)

In [6]:
data = raw_data.splitlines()

In [7]:
class Boat:
    
    def __init__(self):
        
        self.ew = 0
        self.ns = 0
        self.heading = 90
        
    
    def process_instructions(self, instructions):
        
        self.__init__()
        
        for instruction in instructions:
            
            self.process_instruction(instruction)
            
        print(abs(self.ew) + abs(self.ns))
        
    def process_instruction(self, instruction):
        
        preamble = instruction[0]
        
        action = int(instruction[1:])
        
        if preamble == 'F':
            
            heading = self.heading % 360
        
            if heading == 90:
                
                preamble = 'E'
                
            elif heading == 0:
                
                preamble = 'N'
                
            elif heading == 270:
                
                preamble = 'W'
                
            else:
                
                preamble = 'S'
        
        
        match preamble:
            
            case 'N':
                self.ns += action
                
            case 'S':
                self.ns -= action
                
            case 'E':
                self.ew += action
                
            case 'W':
                self.ew -= action
                
            case "R":
                self.heading += action
                
            case "L":
                self.heading -= action
                
                
                
    

In [8]:
boat = Boat()

In [9]:
boat.process_instructions(data)

845


In [11]:
class Boat:
    
    def __init__(self):
        
        self.position = np.array([0,0], dtype=float)
        self.waypoint = np.array([10, 1], dtype=float)
        
    
    def process_instructions(self, instructions, debug=False):
        
        self.__init__()
        
        self.debug = debug
        
        for instruction in instructions:
            
            self.process_instruction(instruction, debug)
            
        print(np.abs(self.position).sum())
        
    def process_instruction(self, instruction, debug):
        
        preamble = instruction[0]
        
        action = int(instruction[1:])
        
        match preamble:
            
            case 'N':
                self.waypoint += np.array([0,1], dtype=float) * action
                
            case 'S':
                self.waypoint += np.array([0, -1], dtype=float) * action
                
            case 'E':
                self.waypoint += np.array([1,0],dtype=float) * action
                
            case 'W':
                self.waypoint += np.array([-1, 0],dtype=float) * action
                
            case "R":
                self.waypoint = self.rotate(action * -1.0, self.waypoint)
                
            case "L":
                self.waypoint = self.rotate(action, self.waypoint)
                
            case "F":
                self.position += self.waypoint * action
        
        if debug:        
            print(self.position, self.waypoint, instruction)
                
    def rotate(self, angle, vector):
        
        theta = np.radians(angle)
        c, s = np.cos(theta), np.sin(theta)
        R = np.array(((c, -s), (s, c)))
        
        return np.dot(R, vector)
        

In [12]:
boat = Boat()

In [13]:
boat.process_instructions(['F10', 'N3', 'F7', 'R90', 'F11'], True)

[100.  10.] [10.  1.] F10
[100.  10.] [10.  4.] N3
[170.  38.] [10.  4.] F7
[170.  38.] [  4. -10.] R90
[214. -72.] [  4. -10.] F11
286.0


In [14]:
boat.process_instructions(data)

27016.00000000003


# DAY 13

In [2]:
raw_data = aoc.get_data(2020, 13)

In [3]:
timestamp, schedule = raw_data.splitlines()

In [38]:
class Bus:
    
    def __init__(self, id):
        self.id = id
        
        
    def get_next_arrival_after_timestamp(self, timestamp):
            
        return self.id - (timestamp % self.id)
        
    def __repr__(self):
        return f'{self.id}'

In [53]:
class Scheduler:
    
    def __init__(self, schedule):
        
        ids = [int(x) for x in schedule.split(',') if x != 'x']
        
        self.buses = [Bus(id) for id in ids]
        
    def get_next_bus(self, timestamp):
        
        delta_arrival_times = [bus.get_next_arrival_after_timestamp(timestamp) for bus in self.buses]
        
        min_wait_time = min(delta_arrival_times)
        
        bus = self.buses[delta_arrival_times.index(min_wait_time)]
        
        return min_wait_time * bus.id

In [54]:
scheduler = Scheduler('7,13,x,x,59,x,31,19')

In [55]:
scheduler.get_next_bus(939)

295

In [56]:
scheduler = Scheduler(schedule)

In [57]:
scheduler.get_next_bus(int(timestamp))

2298

In [48]:
bus = Bus(7)

In [49]:
bus.get_next_arrival_after_timestamp(9)

5

In [153]:
class TimeStampScheduler:
    
    def __init__(self, schedule):
        
        schedule = schedule.split(',')
        
        self.buses = [int(x) for x in schedule if x != 'x']

        self.departures = [i for i, element in enumerate(schedule) if element !='x']
        
        
    def get_timestamp(self):
        
        
        return reduce(lambda a,b: a*b, self.buses) - chinese_remainder(self.buses, self.departures)
        
    
    
    def chinese_remainder(n, a):
        
        sums = 0
        
        prod = reduce(lambda a, b: a*b, n)
        
        for n_i, a_i in zip(n, a):
        
            p = prod // n_i

            sums += a_i * self.mul_inv(p, n_i) * p
        
        return sums % prod

    def mul_inv(a, b):
        
        b0 = b
        
        x0, x1 = 0, 1
        
        if b == 1: return 1
        
        while a > 1:
        
            q = a // b
            
            a, b = b, a%b
            
            x0, x1 = x1 - q * x0, x0
        
        if x1 < 0: x1 += b0
        
        return x1        

In [160]:
scheduler = TimeStampScheduler(schedule)

In [161]:
scheduler.get_timestamp()

783685719679632

# Day 14

In [3]:
raw_data = aoc.get_data(2020, 14)

In [4]:
data = raw_data.splitlines()

In [5]:
class BitComputer:
    
    def __init__(self):
        
        self.memory = dict()
        self.bitmask = None
        
        
    def process_instructions(self, instructions):
        
        for instruction in instructions:
            
            self.process_instruction(instruction)
            
    def process_instruction(self, instruction):
        
        instruction = instruction.split(' = ')
        
        
        if 'mask' in instruction[0]:
            
            self.bitmask = instruction[1]
            
        else:
            
            memory_location = int(re.search(r'\d+', instruction[0])[0])
            
            self.memory[memory_location] = self.apply_mask(instruction[1])
            
    def apply_mask(self, number):
        bit_representation = "{0:b}".format(int(number)).zfill(36)
        
        output_string =''
        
        for value, mask in zip(bit_representation, self.bitmask):
            
            if mask == 'X':
                output_string += value
                
            else:
                output_string += mask
                
        return int(output_string,2)

    def get_mem(self):
        return sum(self.memory.values())

In [6]:
computer = BitComputer()

In [7]:
computer.process_instructions(data)

In [8]:
computer.get_mem()

15018100062885

In [15]:
class BitComputerV2(BitComputer):
    
    
    def __init__(self):
        super().__init__()
        self.value = None
    
    def process_instruction(self, instruction):
        
        instruction = instruction.split(' = ')
        
        
        if 'mask' in instruction[0]:
            
            self.bitmask = instruction[1]
            
        else:
            self.value = int(instruction[1])
            self.get_memory_locations(int(re.search(r'\d+', instruction[0])[0]))

     
    def get_memory_locations(self, number):
        
        bit_representation = "{0:b}".format(int(number)).zfill(36)
        
        floating_memory_locations = ''
        
        for value, mask in zip(bit_representation, self.bitmask):
            
            
            if mask == '0':
                floating_memory_locations += value
                
            elif mask == '1':
            
                floating_memory_locations += '1'
                
            else:
                
                floating_memory_locations += 'X'
                
                
        self.fill_floating_bit(floating_memory_locations)
                
    def fill_floating_bit(self, mem_location):
                
        if 'X' not in mem_location:
            
            self.memory[int(mem_location, 2)] = self.value
        
        else:
            
            self.fill_floating_bit(mem_location.replace('X', '1', 1)), self.fill_floating_bit(mem_location.replace('X', '0', 1))
        

            
        

In [16]:
computer = BitComputerV2()

In [17]:
computer.process_instructions(data)

In [75]:
computer.get_mem()

5724245857696

# Day 15

In [4]:
raw_data = aoc.get_data(2020, 15)

In [5]:
raw_data.strip().split(',')

['1', '20', '11', '6', '12', '0']

In [54]:
class SpokenWord:
    
    def __init__(self, word, turn_spoken):
        
        self.word = word
        self.turn_spoken = turn_spoken
        self.turn_spoken_before = None
        
    def __repr__(self):
        
        return f'word {self.word} : turn {self.turn_spoken} : spoken before {self.turn_spoken_before}'
    
    def __eq__(self, other):
        return self.word == other.word


class Game:
    
    
    def __init__(self, initial_numbers):
        
        self.spoken_words = [SpokenWord(int(x), i + 1) for i, x in enumerate(initial_numbers.strip().split(','))]
        self.turn = len(self.spoken_words) + 1

        
        
    def play(self, stop):
        
        while self.turn <= stop:
            
            self.play_turn()
            self.turn += 1

            
    def play_turn(self):
        
        last_spoken_word = self.spoken_words[-1]
        
        self.say_next_word(last_spoken_word)
        
    def say_next_word(self, last_spoken_word):
        
        if last_spoken_word.turn_spoken_before:
            
            speak_word = SpokenWord(last_spoken_word.turn_spoken-last_spoken_word.turn_spoken_before, self.turn)
            
        else:
            
            speak_word = SpokenWord(0, self.turn)
            
    
            
        if speak_word in self.spoken_words:
            
            previous_time_spoken = self.spoken_words.pop(self.spoken_words.index(speak_word))
            
            speak_word.turn_spoken_before = previous_time_spoken.turn_spoken
        
        self.spoken_words.append(speak_word)
        
    def get_last_word_spoken(self):
        print(self.spoken_words[-1])

In [55]:
game = Game(raw_data)

In [56]:
game.play(2020)

In [57]:
game.get_last_word_spoken()

word 1085 : turn 2020 : spoken before None


In [74]:
class FastGame(Game):
    
    def __init__(self, initial_numbers):
        
        self.last_spoken_word = None
        self.last_turn_word_spoken = dict()
        self.turn_before_last_turn_word_spoken = dict()
        self.turn = 1
        
        for word in initial_numbers.strip().split(','):
            
            self.last_spoken_word = int(word)
            self.last_turn_word_spoken[self.last_spoken_word] = self.turn
            
            self.turn += 1
        
    def play_turn(self):
        
        last_turn_spoken = self.last_turn_word_spoken.get(self.last_spoken_word)
        turn_before_last_turn_spoken = self.turn_before_last_turn_word_spoken.get(self.last_spoken_word)
        
        if last_turn_spoken and turn_before_last_turn_spoken:
            
            self.say_word(last_turn_spoken - turn_before_last_turn_spoken)
            
        else:
            
            self.say_word(0)
            
        
    def say_word(self, word):
        
        self.last_spoken_word = word
        
        last_turn_spoken = self.last_turn_word_spoken.get(word)
        
        self.turn_before_last_turn_word_spoken[word] = last_turn_spoken
        self.last_turn_word_spoken[word] = self.turn
        
    
    def get_last_word_spoken(self):
        print(self.last_spoken_word)

In [75]:
game = FastGame(raw_data)

In [76]:
game.play(30000000)
game.get_last_word_spoken()

10652


# Day 16

In [2]:
raw_data = aoc.get_data(2020, 16)

In [77]:
class TicketValidator():
    
    def __init__(self, rules):
        
        self.rules = [Rule(x) for x in rules]
            

    def check_ticket(self, ticket):

        bad_values = list()
        
        for value in ticket.split(','):
        
            value = int(value)
            
            if all([x.check_bad(value) for x in self.rules]):
                bad_values.append(value)
                       
        return sum(bad_values)
    

    def check_tickets(self, tickets):
        
        return sum([self.check_ticket(ticket) for ticket in tickets])
        
    
class Rule:
    
    def __init__(self, rule_string):
        
        name, rule_values = rule_string.split(': ')
        
        self.name = name
        
        first_range, second_range = rule_values.split(' or ')
        
        self.values = self.create_range(first_range) + self.create_range(second_range)
        
        self.column = set()
        
    def create_range(self, range_string):
        
        first, last = range_string.split('-')
        
        return list(range(int(first), int(last) + 1))
        
        
    def check_bad(self, value):
        
        return not value in self.values
        
    def __repr__(self):
        return f'{self.name}'

In [5]:
validator = TicketValidator("""class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50""".splitlines())

In [6]:
validator.check_tickets("""7,3,47
40,4,50
55,2,20
38,6,12""".splitlines())

71

In [104]:
rules, my_ticket, other_tickets = raw_data.split('\n\n')

In [8]:
validator = TicketValidator(rules.splitlines())

In [9]:
validator.check_tickets(other_tickets.splitlines()[1:])

25984

In [157]:
class TicketPropertyFinder(TicketValidator):
    
    def __init__(self, rules):
        
        super().__init__(rules)
        self.valid_tickets = None
    
    @property
    def departure_rules(self):
    
        return [x for x in self.rules if 'departure' in x.name]
    
    def sort_rules(self, ticket):
        
        check = map(int, ticket.split(","))
        
        for i, value in enumerate(check):
            
            column = self.get_column(i)
            
            for rule in self.rules:
                
                rule_applied = [not rule.check_bad(x) for x in column]
            
                if all(rule_applied):
                    rule.column.add(i)
    
        self.rules = sorted(self.rules, key=lambda x: x.column)
        
        
    def reduce_rules(self):
        
        for i, rule in enumerate(self.rules[:-1]):
            
            value = next(iter(rule.column))
            
            rule.column = value
            
            for other_rule in self.rules[i+1:]:
                
                other_rule.column.remove(value)
            
        last_rule = self.rules[-1]
        
        last_rule.column = next(iter(last_rule.column))
                
            
            
    def score_ticket(self, ticket):
        
        
        values = list(map(int, ticket.split(',')))
        
        self.sort_rules(ticket)
        self.reduce_rules()
        
        
        return reduce(lambda a,b: a*b, [values[rule.column] for rule in self.departure_rules])
    
                    
    def get_column(self, i):
        
        
        return [x[i] for x in self.valid_tickets]
    
    
    def check_ticket(self, ticket):
        
        for value in ticket.split(','):
            
            value = int(value)
            
            if all([x.check_bad(value) for x in self.rules]):
                return
            
        return ticket
    
    def check_tickets(self, tickets):
        
        
        valid_tickets = list()
        
        for ticket in tickets:
        
            valid_ticket = self.check_ticket(ticket)
            
            if valid_ticket:
                
                valid_tickets.append(list(map(int, valid_ticket.split(','))))
        
        self.valid_tickets = valid_tickets
        

In [158]:
validator = TicketPropertyFinder(rules.splitlines())

In [159]:
validator.check_tickets(other_tickets.splitlines()[1:])

In [160]:
validator.score_ticket(my_ticket.splitlines()[1])

1265347500049

# Day 17

In [15]:
raw_data = aoc.get_data(2020, 17)

In [7]:
class Field:
    
    def __init__(self, input_field):
        
        width = len(input_field.splitlines()[0])
        height = len(input_field.splitlines())
        
        values = np.fromstring(','.join(input_field.replace('\n', '')).replace('.', '0').replace('#', '1'), sep=',', dtype=int).reshape((width,height))
        
        self.image = np.zeros((len(values[0]),len(values), 1), dtype=int)
        self.image[:,:, 0] = values
        self.kernel = np.ones((3,3,3), dtype=int)
        self.kernel[1,1,1] = 0

    def iterate(self):
        
        self.image = np.pad(self.image, 1, mode='constant', constant_values=0)
        
        convolution = convolve(self.image, self.kernel, mode='constant', cval=0)
        
        three_neigbhors = convolution == 3
        two_neigbhors = convolution == 2
        
        remain_lit = (three_neigbhors ^ two_neigbhors) & (self.image == 1)
        light_up = three_neigbhors & (self.image == 0)
        
        self.image = np.zeros(self.image.shape, dtype=int) + remain_lit + light_up
        
    def simulate(self, steps=6):
        
        for i in range(steps):
            self.iterate()
            
    def count_lit(self):
        return self.image.sum()

In [8]:
field = Field(raw_data)

In [9]:
field.simulate(6)

In [10]:
field.count_lit()

395

In [16]:
class Tesseract(Field):
    
    def __init__(self, input_field):

        width = len(input_field.splitlines()[0])
        height = len(input_field.splitlines())
        
        values = np.fromstring(','.join(input_field.replace('\n', '')).replace('.', '0').replace('#', '1'), sep=',', dtype=int).reshape((width,height))
        
        self.image = np.zeros((len(values[0]),len(values), 1, 1), dtype=int)
        self.image[:,:, 0, 0] = values
        self.kernel = np.ones((3,3,3,3), dtype=int)
        self.kernel[1,1,1,1] = 0

In [17]:
tesseract = Tesseract(raw_data)

In [18]:
tesseract.simulate(6)

In [19]:
tesseract.count_lit()

2296

# Day 18

In [2]:
raw_data = aoc.get_data(2020, 18)

In [94]:
class EqualEvaluator:

    def split_equation(self, equation):
        
        return re.findall('[0-9+*]+|[()]', equation)
        
    def gobble(self, equation):
        
        if equation[0] in '+*()':
            return equation
        
        values = re.findall('\d+|[*+]', equation)
        
        while len(values) > 2:
           
            left = values.pop(0)
            operator = values.pop(0)
            right = values.pop(0)
            
            evaluation = eval(f"{left}{operator}{right}")
            
            values.insert(0, f"{evaluation}")
        
        return ''.join(values)
            
    def reduce_equation(self, equation):
        
        equation = equation.replace(" ", "")
        
        while not re.match('^\d+$', equation):
            
            split = self.split_equation(equation)
            
            equation = ''.join(map(self.gobble, split))
            
            equation = re.sub('\((\d+)\)', lambda match: match.group(1), equation)
            
        return int(equation)
    
    def evaluate(self, equations):
        return sum(map(self.reduce_equation, equations))

In [95]:
evaluator = EqualEvaluator()
evaluator.evaluate(raw_data.splitlines())

1408133923393

In [96]:
class AdditionFirstEvaluator(EqualEvaluator):

    def wrap_addition(self, equation):
        return re.sub('(\d+\+\d+)', lambda match: f"({match.group(1)})", equation)
    
    def reduce_equation(self, equation):
        
        equation = equation.replace(" ", '')
        
        while not re.match('^\d+$', equation):
            
            equation = self.wrap_addition(equation)
            
            split = self.split_equation(equation)
            
            equation = ''.join(map(self.gobble, split))
            
            equation = re.sub('\((\d+)\)', lambda match: match.group(1), equation)
            
        return int(equation)
            
    def gobble(self, equation):
        
        if equation[-1] in '+*':
            return equation
        
        return super().gobble(equation)

        


In [97]:
evaluator = AdditionFirstEvaluator()
evaluator.evaluate(raw_data.splitlines())

314455761823725

# Day 19

In [2]:
raw_data = aoc.get_data(2020,19)

In [71]:
class MessageValidator:
    
    def __init__(self, text_rules):
        
        self.rules = dict()
        
        for rule in text_rules.splitlines():
            
            rule_index, rule_values = rule.split(':')
            
            self.rules[rule_index] = rule_values.strip().replace('"', '').split(' ')
            
        self.valid_strings = self.make_valid_strings()
    
    def make_valid_strings(self):
        
        return "^" + "".join(map(self.reduce_rule, self.rules['0'])) + "$"
        
        
    def reduce_rule(self, element):
        
        next_rule = self.rules.get(element)
        
        if next_rule is None:
            return element
        
        resolved = ''.join(map(self.reduce_rule, next_rule))
        
        if '|' in resolved:
            return '(?:' + resolved + ')'
        
        else: 
            return resolved
        
        
    def validate_message(self, message):
        
        match = re.match(self.valid_strings, message)
        
        if match:
            return True
        
        else:
            return False
        
    def count_valid_messages(self, messages):
        
        return sum(map(self.validate_message, messages.splitlines()))

In [132]:
validator = MessageValidator("""0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b""")

In [133]:
validator.valid_strings

'a(?:(?:aa|bb)(?:ab|ba)|(?:ab|ba)(?:aa|bb))b$'

In [87]:
validator.count_valid_messages("""ababbb
bababa
abbbab
aaabbb
aaaabbb""")

2

In [88]:
rules, messages = raw_data.split('\n\n')

In [89]:
validator = MessageValidator(rules)

In [90]:
validator.valid_strings

'(?:(?:a(?:(?:b(?:a(?:(?:a(?:ba|ab)|b(?:bb|aa))b|(?:a(?:aa|b(?:a|b))|bba)a)|b(?:(?:bab|(?:a(?:a|b)|ba)a)b|(?:baa|(?:aa|ba)b)a))|a(?:(?:b(?:bab|(?:a(?:a|b)|ba)a)|a(?:(?:aa|b(?:a|b))a|(?:(?:a|b)(?:a|b))b))b|(?:(?:a(?:ab|aa)|b(?:ab|bb))a|(?:baa|bab)b)a))b|(?:a(?:a(?:(?:b(?:aa|ba))a|(?:b(?:a(?:a|b)|ba)|aaa)b)|b(?:b(?:a(?:aa|ba)|b(?:ba|ab))|a(?:bba|aaa)))|b(?:(?:a(?:(?:(?:a|b)(?:a|b))b|(?:bb|aa)a)|b(?:(?:a|b)(?:aa|ba)))b|(?:b(?:(?:ab|bb)b|(?:(?:a|b)(?:a|b))a)|a(?:bab|(?:a(?:a|b)|ba)a))a))a)|b(?:(?:a(?:(?:b(?:a(?:aa|b(?:a|b))|b(?:aa|(?:a|b)b))|a(?:aba|b(?:aa|(?:a|b)b)))a|(?:b(?:aba|b(?:aa|(?:a|b)b))|a(?:(?:a|b)(?:ab|bb)))b)|b(?:b(?:(?:a(?:aa|b(?:a|b))|b(?:aa|(?:a|b)b))b|(?:(?:aa|ba)b|(?:a(?:a|b)|ba)a)a)|a(?:b(?:b(?:aa|ba)|a(?:ba|bb))|a(?:b(?:a(?:a|b)|ba)|aaa))))a|(?:b(?:a(?:(?:a|b)(?:b(?:ab|bb)|aba))|b(?:a(?:aba|b(?:ab|aa))|b(?:(?:aa|ba)b)))|a(?:b(?:a(?:(?:ab|aa)b|(?:ba|bb)a)|b(?:b(?:bb|aa)))|a(?:(?:a(?:aa|b(?:a|b))|b(?:ab|bb))a|(?:b(?:aa|b(?:a|b))|a(?:ab|bb))b)))b)))(?:(?:a(?:(?:b(?:a(?:(?:

In [91]:
validator.count_valid_messages(messages)

182

# Day 20

In [92]:
raw_data = aoc.get_data(2020, 20)

In [125]:
class Picture:
    
    def __init__(self, data):
        self.tiles = [Tile(x) for x in data.strip().split('\n\n')]
        
        self.datum = self.tiles[0]
        
        self.datum.position = np.array([0,0])

    def sort_tiles(self):
        
        while self.unsorted():
            
            for unsorted_tile, sorted_tile in itertools.product(self.unsorted(), self.sorted()):
                
                top, bottom, left, right = sorted_tile.get_edges()
                
                
                
            
            
    def unsorted(self):
        return [x for x in self.tiles if not x.position]
    
    def sorted(self):
        return [x for x in self.tiles if x.position]
        
class Tile:
    
    def __init__(self, data):
        
        id, *image = data.splitlines()
        
        self.id = int(re.match('Tile (\d+):', id).group(1))
        
        image = '\n'.join(image)
        
        width = len(image.splitlines()[0])
        height = len(image.splitlines())
        
        self.image = np.fromstring(','.join(image.replace('\n', '')).replace('.', '0').replace('#', '1'), sep=',', dtype=int).reshape((width,height))
            
        self.position = None

    def get_edges(self):
        # top, bottom, left, right
        return self.image[0, :], self.image[-1, :], self.image[: , 0], self.image[: , -1]
        

    def rotate(self):
        self.image = np.rot90(self.image)

    def flip(self):
        
        self.image = np.flipud(self.image)

    def __repr__(self):
        return f"Tile {self.id}"
        
        

In [126]:
picture = Picture(raw_data)

In [127]:
picture.tiles[0].get_edges()

(array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 1, 0, 0, 0, 1]),
 array([0, 1, 1, 1, 0, 1, 1, 1, 0, 0]),
 array([0, 1, 0, 0, 0, 0, 0, 0, 0, 1]))

In [128]:
picture.tiles[0].image

array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 1, 0, 1, 1],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 1, 1, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 1]])

In [131]:
list(itertools.product('ab', ''))
    
    

[]